In [4]:
from qiskit import IBMQ

IBMQ.save_account("Please replace with your own token.", overwrite=True)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2023-09-16 19:01:18,482: Credentials are already in use. The existing account in the session will be replaced.


In [5]:
from helper import generate_2d_grid_coupling
from quantum_circuit_generator.generators import gen_supremacy, gen_uccsd,  gen_qft, gen_BV, gen_adder,  gen_hwea
from qiskit import QuantumCircuit
from qiskit.compiler import transpile

prefix = 'exper_result/230916/testing'
expr_name = 'grami_approx'
fname = 'test'

config = {'min_freq': 8, 
            'prune_method': 'critical', #default, c_c
            'topk': 5, 
            'maxN': 3, 
            'merge_heuristic': 'approx', #random, approx2
            'proctect_pattern': False, 
            'merge_cont1q': True, 
            'merge_cont2q': True, 
            'enable_grami': True}

simulator_backend = provider.get_backend('ibmq_qasm_simulator')
coupling_map = generate_2d_grid_coupling(5)
# circ = gen_adder(nbits=8,a=0, b=0, use_toffoli=True, regname='q', barriers=False)
circ = QuantumCircuit.from_qasm_file('Benchmarks/test/test.qasm')

output_file = f'{prefix}_{expr_name}_{fname}.txt'
print(f'name: {fname}\ninit:nqubits: {circ.num_qubits}, ngates: {circ.size()}')
print(output_file)

circ = transpile(circ, 
                coupling_map=coupling_map, backend=simulator_backend, 
                layout_method='sabre',
                routing_method = 'sabre',
                seed_transpiler = 2021,
                optimization_level = 0)
print(f'compiled:nqubits: {circ.num_qubits}, ngates: {circ.size()}')

name: test
init:nqubits: 5, ngates: 40
exper_result/230916/testing_grami_approx_test.txt
compiled:nqubits: 25, ngates: 43


In [7]:
from paqoc import PAQOC
import pickle

paqoc = PAQOC(fname,
                circ,
                min_freq=config['min_freq'], 
                prune_method=config['prune_method'], 
                enable_grami = config['enable_grami'],
                proctect_pattern = config['proctect_pattern'],
                merge_cont1q = config['merge_cont1q'],
                merge_cont2q = config['merge_cont2q'],
                merge_heuristic = config['merge_heuristic'], 
                topk = config['topk'], 
                maxN = config['maxN'])
paqoc.run()
paqoc.save_to_pickle(output_file)
with open(output_file, 'rb') as fp:
    all_results = pickle.load(fp)
    print(all_results)

	Convert dag to grami graph 0.0036
	Upload input graph 0.3853
	Running GRAMI. 0.6310
	Generating frequent subcircuits from grami output. 0.0238
nodes:
	 (0, 'cx')
	 (1, 'cx')
	 (2, 'rz')
edges:
	 (0, 1, '0.0')
	 (0, 2, '1.0')
	 (2, 1, '0.1')
	Convert dag to grami graph 0.0025
	Upload input graph 0.3677
	Running GRAMI. 0.3534
	Generating frequent subcircuits from grami output. 0.0046
iter: 1
	# candidates: 5
	calculate scores...Finish

	Merge 1 candidates...
	Merge 2 candidates...
	Merge 3 candidates...
	Merge 3 candidates...
	Merge 4 candidates...
iter: 1, steps: 50 / [40-60] total_time: 25.0
25.0 0.000993967
iter: 2, steps: 45 / [40-50] total_time: 22.5
22.5 0.0009945035
iter: 3, steps: 42 / [40-45] total_time: 21.0
21.0 0.0009908676
iter: 4, steps: 41 / [40-42] total_time: 20.5
20.5 0.0009977818
iter: 1, steps: 30 / [20-40] total_time: 15.0
15.0 0.0029233694
iter: 2, steps: 35 / [30-40] total_time: 17.5
17.5 0.003305912
iter: 3, steps: 37 / [35-40] total_time: 18.5
18.5 0.00099653
it